# Chapter 8

In [2]:
%%sql
use prod.db;

++
||
++
++

In [3]:
%%sql
SELECT
    l_orderkey,
    round( sum(l_extendedprice * (1 - l_discount) * (1 + l_tax)),
        2
    ) AS totalprice
FROM
    lineitem
WHERE
    l_orderkey = 1
GROUP BY
    l_orderkey;


l_orderkey,totalprice
1,194029.59


In [4]:
%%sql

-- The totalprice of an order (with orderkey = 1)
SELECT
    o_orderkey,
    o_totalprice
FROM
    orders
WHERE
    o_orderkey = 1;



o_orderkey,o_totalprice
1,194029.55


In [5]:
%%sql
    CREATE TABLE dim_customer AS 
 SELECT 
        c.*,
        n_name AS nation_name,
        n_comment AS nation_comment,
        r_name AS region_name,
        r_comment AS region_comment
    FROM customer c
    LEFT JOIN nation n ON c_nationkey = n_nationkey
    LEFT JOIN region r ON n_regionkey = r_regionkey

++
||
++
++

In [6]:
%%sql
CREATE TABLE fct_orders as select * from orders

++
||
++
++

In [7]:
%%sql
CREATE TABLE fct_lineitem as select * from lineitem

++
||
++
++

In [8]:
%%sql
create table wide_orders as 
 SELECT 
        f.*,
        d.*
    FROM fct_orders f
    LEFT JOIN dim_customer d ON f.o_custkey = d.c_custkey

++
||
++
++

In [9]:
%%sql
create table wide_lineitem as select * from fct_lineitem

++
||
++
++

In [10]:
%%sql
create table order_lineitem_metrics as 
SELECT 
        l_orderkey as order_key,
        COUNT(l_linenumber) AS num_lineitems
    FROM wide_lineitem
    GROUP BY l_orderkey

++
||
++
++

In [11]:
%%sql
create table customer_outreach_metrics as 
SELECT 
        wo.c_custkey,
        wo.c_name AS customer_name,
        MIN(wo.o_totalprice) AS min_order_value,
        MAX(wo.o_totalprice) AS max_order_value,
        AVG(wo.o_totalprice) AS avg_order_value,
        AVG(olm.num_lineitems) AS avg_num_items_per_order
    FROM wide_orders wo
    LEFT JOIN order_lineitem_metrics olm ON wo.o_orderkey = olm.order_key
    GROUP BY wo.c_custkey, wo.c_name

++
||
++
++

In [27]:
%%sql --show
select *
from customer_outreach_metrics

c_custkey,customer_name,min_order_value,max_order_value,avg_order_value,avg_num_items_per_order
13678,Customer#000013678,3643.59,447542.26,150829.488571,3.6666666666666665
12484,Customer#000012484,3871.10,261383.06,122749.683636,3.5454545454545454
7940,Customer#000007940,53746.57,320282.96,165889.345455,4.909090909090909
4945,Customer#000004945,2205.73,276173.48,127629.438947,3.8421052631578947
13276,Customer#000013276,58779.29,300437.66,149012.655238,4.0
12814,Customer#000012814,20503.71,334985.12,156526.686842,4.526315789473684
12106,Customer#000012106,12701.69,322913.43,167069.977500,4.375
12553,Customer#000012553,20988.22,301496.28,164803.370000,4.428571428571429
2608,Customer#000002608,3455.62,249395.34,109330.526957,3.739130434782609
6793,Customer#000006793,9423.28,341434.58,154404.379167,4.0


25/07/05 11:04:13 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 35210824 ms exceeds timeout 120000 ms
25/07/05 11:04:13 WARN SparkContext: Killing executors is not supported by current scheduler.
25/07/05 11:04:13 WARN Executor: Issue communicating with driver in heartbeater
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:101)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:85)
	at org.apache.spark.storage.BlockManagerMaster.registerBlockManager(BlockManagerMaster.scala:80)
	at org.apache.spark.storage.BlockManager.reregister(BlockManager.scala:642)
	at org.apache.spark.executor.Executor.reportHeartBeat(Executor.scala:1223)
	at